# Evaluation rerun

the evaluation failed for some for questions, but we have the model answers so I want to rerun the evaluation

In [ ]:
#| default_exp rerun_eval

In [ ]:
#| export
from cat_tools.eval import similarity_scorer, conf_variants, prepare_declarative_memory, CatModel
import weave
import pandas as pd
from tqdm.auto import tqdm
import polars as pl
from polars import col as c
from cat_tools.client import SuperCatClient
from datetime import datetime

In [ ]:
#| export
weave_client = weave.init("smart-drying-unitus/declarative-eval")

weave: weave version 0.52.6 is available!  To upgrade, please run:
weave:  $ pip install weave --upgrade
weave: Logged in as Weights & Biases user: smassaro-unitus.
weave: View Weave data at https://wandb.ai/smart-drying-unitus/declarative-eval/weave


In [ ]:
#| export
calls_query={"$expr":{"$and":[{"$gt":[{"$getField":"started_at"},{"$literal":1756801780.338}]},{"$or":[{"$eq":[{"$getField":"id"},{"$literal":"019927e9-0d45-7a20-a70c-ad6886ffdb46"}]},{"$eq":[{"$getField":"id"},{"$literal":"019926be-3e2c-7041-9927-26b76aa3fdb9"}]},{"$eq":[{"$getField":"id"},{"$literal":"019925fa-39ee-7f57-b923-eb9cdb36c1e6"}]},{"$eq":[{"$getField":"id"},{"$literal":"0199175a-03ae-7809-8292-f22c2320f1ee"}]},{"$eq":[{"$getField":"id"},{"$literal":"01990fbd-ae07-74cc-9b8e-9e5ae323d958"}]},{"$eq":[{"$getField":"id"},{"$literal":"019910d8-995a-78ed-8151-afbd5a5e4223"}]},{"$eq":[{"$getField":"id"},{"$literal":"0199115d-6ac3-7422-815a-c2b56b7decae"}]},{"$eq":[{"$getField":"id"},{"$literal":"01991448-bdfd-762d-ab6c-c4851521d521"}]},{"$eq":[{"$getField":"id"},{"$literal":"01991061-7220-7a2a-aa03-a14bd6c8a003"}]}]}]}}

In [ ]:
#| export
def parse_calls(calls):
    outs = []
    for call in tqdm(list(calls)):
        model_name = call.inputs['model'].name
        has_mem =  'NO memory' not in call.display_name
        for row in call.children():
            out = {}
            if inp := row.inputs.unwrap():
                if inp := inp.get('example', False):
                        out['question_id']= inp['id']
                        out['question']= inp['prompt']
                        out['expected_answer']= inp['target']
            if output := row.output:
                output = output.unwrap()
                if 'output' in output:
                    if 'text_clean' in output['output']:
                        out['model_answer'] = output['output']['text_clean']
                    if 'scores' in output:
                        if 'CatEmbeddingSimilarityScorer' in output['scores']:
                            out['similarity_score'] = output['scores']['CatEmbeddingSimilarityScorer']['similarity_score']

            out['model_name'] = model_name
            out['weave_call'] = row
            out['has_declarative_memory'] = has_mem
            outs.append(out)
    return pd.DataFrame(outs)

In [ ]:
calls = weave_client.get_calls(
    filter={"trace_roots_only": True},
    query = calls_query,
    sort_by=[{"field":"started_at","direction":"desc"}],
)

In [ ]:
df = parse_calls(calls)

  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 

weave: 🍩 https://wandb.ai/smart-drying-unitus/declarative-eval/r/call/01992df7-3180-7fec-8e3c-407c90711749
INFO:weave.trace.weave_client:🍩 https://wandb.ai/smart-drying-unitus/declarative-eval/r/call/01992df7-3180-7fec-8e3c-407c90711749


In [ ]:
df

,question_id,question,expected_answer,model_name,weave_call,has_declarative_memory,model_answer,similarity_score
0,0.0,Che cos'è la disidratazione a flusso di aria c...,La disidratazione a flusso di aria calda è un ...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,False,NaN,NaN
1,1.0,Qual è l'obiettivo principale del processo di ...,L'obiettivo principale del processo di disidra...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,False,NaN,NaN
2,2.0,Quali sono i principali parametri da controlla...,Durante la disidratazione a flusso di aria cal...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,False,NaN,NaN
3,3.0,Quali fenomeni fisici avvengono durante la dis...,Durante la disidratazione di un alimento avven...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,False,NaN,NaN
4,4.0,Quali vantaggi offre la disidratazione oltre a...,"Oltre alla conservazione, la disidratazione of...",deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,False,NaN,NaN
...,...,...,...,...,...,...,...,...
3201,332.0,Come vengono caricati e scaricati i contenitor...,I contenitori del prodotto vengono caricati e ...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True,I contenitori vengono caricati e scaricati att...,0.983811
3202,333.0,In che modo questo essiccatore gestisce il ric...,Questo essiccatore gestisce il ricircolo dell'...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True,Gestisce il ricircolo tramite un passaggio ret...,0.924412
3203,334.0,Perché le pareti dell'essiccatore sono realizz...,Le pareti dell'essiccatore sono realizzate con...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True,"Per garantire isolamento, efficienza e durata....",0.892559
3204,335.0,È possibile impostare diverse temperature per ...,"Sì, il pannello di controllo di questo essicca...",openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True,Sì. Il pannello di controllo consente di impos...,0.956343


In [ ]:
call = calls[0]

In [ ]:
call.display_name

'09-03 15:20 Eval - deepseek_best NO memory'

In [ ]:
df_pl = pl.from_pandas(df.drop(columns=['weave_call']))

In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory).agg(frac_answers = c.model_answer.is_not_null().sum() / 336, frac_scores = c.similarity_score.is_not_null().sum() / 336).sort(c.has_declarative_memory, c.model_name)

model_name,has_declarative_memory,frac_answers,frac_scores
str,bool,f64,f64
"""deepseek_best""",false,0.0,0.0
"""gemini_best""",false,0.910714,0.910714
"""gemma_best""",false,0.0,0.0
"""openai_best""",false,0.404762,0.404762
"""qwen_best""",false,0.0,0.0
"""deepseek_best""",true,0.892857,0.892857
"""gemini_best""",true,0.991071,0.991071
"""gemma_best""",true,0.997024,0.997024
"""openai_best""",true,0.985119,0.985119


In [ ]:
missing = df_pl.filter(c.model_answer.is_null()).group_by(c.model_name, c.has_declarative_memory).agg(question = c.question, target = c.expected_answer, id = c.question_id.cast(pl.Int64), n = pl.len()).sort(c.has_declarative_memory)

In [ ]:
missing

model_name,has_declarative_memory,question,target,id,n
str,bool,list[str],list[str],list[i64],u32
"""gemma_best""",false,"[""Che cos'è la disidratazione a flusso di aria calda applicata agli alimenti?"", ""Qual è l'obiettivo principale del processo di disidratazione?"", … null]","[""La disidratazione a flusso di aria calda è un processo tecnologico che consiste nell'applicare calore a un alimento in condizioni controllate. Lo scopo è rimuovere gran parte dell'acqua contenuta al suo interno tramite evaporazione, trasformandola in vapore acqueo che viene poi allontanato."", ""L'obiettivo principale del processo di disidratazione è la conservazione dell'alimento. Riducendo il contenuto di acqua, si abbassa l'attività dell'acqua (aw), un parametro che inibisce la crescita di microrganismi (come batteri, lieviti e muffe) e rallenta le reazioni enzimatiche che causano il deterioramento del prodotto, estendendone così la durata di conservazione (shelf-life)."", … null]","[0, 1, … null]",337
"""openai_best""",false,"[""Che cos'è la Computer Vision (CV) in parole semplici?"", ""Quali sono le principali cause chimiche della diminuzione del WI e come si prevengono?"", … null]","[""La Computer Vision è una branca dell'intelligenza artificiale che ha lo scopo di dare ai computer la capacità di 'vedere' e interpretare il mondo visivo. Utilizzando una telecamera e algoritmi software, un sistema di CV può estrarre, analizzare e comprendere informazioni utili da un'immagine o da un video, in modo automatico."", ""Le principali cause chimiche della diminuzione del WI sono le reazioni di imbrunimento, sia enzimatico (causato da enzimi come la polifenolossidasi) sia non enzimatico (Reazione di Maillard). Si prevengono rispettivamente con pre-trattamenti specifici (es. scottatura, acidificazione) per inattivare gli enzimi e con un attento controllo della temperatura di processo per limitare la reazione di Maillard."", … null]","[100, 137, … null]",201
"""qwen_best""",false,"[""Che cos'è la disidratazione a flusso di aria calda applicata agli alimenti?"", ""Qual è l'obiettivo principale del processo di disidratazione?"", … null]","[""La disidratazione a flusso di aria calda è un processo tecnologico che consiste nell'applicare calore a un alimento in condizioni controllate. Lo scopo è rimuovere gran parte dell'acqua contenuta al suo interno tramite evaporazione, trasformandola in vapore acqueo che viene poi allontanato."", ""L'obiettivo principale del processo di disidratazione è la conservazione dell'alimento. Riducendo il contenuto di acqua, si abbassa l'attività dell'acqua (aw), un parametro che inibisce la crescita di microrganismi (come batteri, lieviti e muffe) e rallenta le reazioni enzimatiche che causano il deterioramento del prodotto, estendendone così la durata di conservazione (shelf-life)."", … null]","[0, 1, … null]",337
"""gemini_best""",false,"[""L'attività dell'acqua (aw) del prodotto finito ha un ruolo nella degradazione dei caroteni durante la conservazione?"", ""In che modo la combinazione di temperatura e tempo di processo influisce sulla conservazione dei pigmenti?"", … null]","[""Sì, l'attività dell'acqua (aw) del prodotto finito ha un ruolo importante. Valori di aw non ottimali possono influenzare la mobilità dei reagenti e accelerare le reazioni. In particolare, mentre un'aw molto bassa è generalmente protettiva, in alcuni intervalli intermedi può favorire l'ossidazione, soprattutto se catalizzata da enzimi residui. Mantenere l'aw nel range di massima stabilità è quindi un punto critico di controllo per la conservazione del colore e del valore nutrizionale."", ""La conservazione dei pigmenti è influenzata dall'equilibrio tra temperatura e tempo di processo, poiché il degrado termico è cumulativo. L'obiettivo è minimizzare lo stress termico complessivo: tempi di processo più brevi e temperature di essiccazione più basse riducono l'esposizione del prodotto al calore e all'ossigeno, limitando così l'ossidazione e la degradazio

In [ ]:
#| export
def get_dataset(missing):
    datasets = []
    for row in missing.iter_rows(named=True):
        dataset = []
        for id, q, t in zip(row['id'], row['question'], row['target']):
            if id is None:
                continue
            dataset.append({
                'id': id,
                'prompt': q,
                'input': q,
                'target': t
            })
        datasets.append(dataset)
    return datasets

In [ ]:
missing = missing.with_columns(dataset = pl.Series(get_dataset(missing), dtype=pl.Object))

In [ ]:
#| export
def get_missing_df(weave_client, query = calls_query):
    calls = list(weave_client.get_calls(
        filter={"trace_roots_only": True},
        query=query,
        sort_by=[{"field":"started_at","direction":"desc"}],
    ))
    df = parse_calls(calls)
    df_pl = pl.from_pandas(df.drop(columns=['weave_call']))
    missing = df_pl.filter(c.model_answer.is_null()).group_by(c.model_name, c.has_declarative_memory).agg(question = c.question, target = c.expected_answer, id = c.question_id.cast(pl.Int64), n = pl.len()).sort(c.has_declarative_memory)
    missing = missing.with_columns(dataset = pl.Series(get_dataset(missing), dtype=pl.Object))
    return missing

In [ ]:
#| export
async def run_missing(missing):
    client = SuperCatClient()
    time = datetime.now().strftime("%m-%d %H:%M")
    eval_name = f"{time} Eval"
    cat_has_memory = False
    for row in missing.iter_rows(named=True):
        dataset = row['dataset']
        model_name = row['model_name']
        has_mem = row['has_declarative_memory']
        if cat_has_memory != has_mem:
            if has_mem:
                print("Preparing declarative memory...")
                prepare_declarative_memory(client)
            else:
                client.wipe_declarative_memory()
            cat_has_memory = has_mem
        print(f"Running {model_name} with memory={has_mem} on {len(dataset)} examples")
        model = CatModel(model_name, conf_variants[model_name], has_declarative_memory=has_mem)
        evaluation = weave.Evaluation(
            dataset=dataset,
            scorers=[similarity_scorer],
            name=eval_name,
        )
        await evaluation.evaluate(
            model, __weave={"display_name": f"{eval_name} - {model_name} {'memory' if has_mem else 'NO memory'}"}
        )

In [ ]:
await run_missing(missing.head(1))

Running gemma_best with memory=False on 336 examples
